In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -U sentence-transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 6.7 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 6.6 MB 41.6 MB/s 
     |████████████████████████████████| 79 kB 4.7 MB/s 
     |████████████████████████████████| 1.2 MB 12.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=1b1f09e8216174a3ea100aca4aa3ef78570f0e95cf10fb8723ec5a6c1f3bf683
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [2]:
from google.colab import drive
drive.mount('/content/drive')
% cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [3]:
import torch
import torch.nn as nn
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import pickle
import json
import os
import pandas as pd
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatiser = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
# don't convert to lowercase as using bert base cased (case is important)
def preprocess(text):
    res = re.sub(r'[^\w\s]', '',text) # Remove punctuations
    res = " ".join(res.split())       # Remove whitespaces
    tokens = word_tokenize(res) 
    new_tokens = [lemmatiser.lemmatize(i) for i in tokens if not i in stop_words] # Remove stopwords and lemmatise
    res = ' '.join(new_tokens)
    return res

In [6]:
with open("eval_texts.json", "r") as fp:
  eval_texts = json.load(fp)
for i in eval_texts:
    eval_texts[i] = preprocess(eval_texts[i])
def evaluate(name, label):
  df = pd.read_csv(name)
  l1 = []
  l2 = []
  l3 = []
  for index,row in df.iterrows():
      id1,id2 = row['pair_id'].split('_')
      if id1 in eval_texts and id2 in eval_texts:
         l1.append(eval_texts[id1])
         l2.append(eval_texts[id2])
         l3.append((row[label]-1)/3)
  di = {'Text 1':l1, 'Text 2':l2, 'GT': l3}
  df = pd.DataFrame(di)
  test_examples=[]
  score=[]
  for i in range(len(df)):
    test_examples.append([df["Text 1"][i], df["Text 2"][i]])
    score.append(df["GT"][i])
    
  return(test_examples,score)

In [12]:
dict = {}
all_labels = ["GEO","ENT","TIME","NAR","Overall","STYLE","TONE"]

In [ ]:

data,score=evaluate("final_evaluation_data.csv", all_labels[0])
model1=pickle.load(open("/content/drive/MyDrive/Weights_final/Geography/bert-base-multilingual-cased_8_2e-05_MSE_6", 'rb'))
scores_predicted=model1.predict(data)

dict[all_labels[0]] = scores_predicted

data,score=evaluate("final_evaluation_data.csv", all_labels[1])
model1=pickle.load(open("/content/drive/MyDrive/Weights_final/Entities/bert-base-multilingual-cased_8_2e-05_MSE_2", 'rb'))
scores_predicted=model1.predict(data)

dict[all_labels[1]] = scores_predicted

data,score=evaluate("final_evaluation_data.csv", all_labels[2])
model1=pickle.load(open("/content/drive/MyDrive/Weights_final/Time/bert-base-multilingual-cased_8_2e-05_MSE_6", 'rb'))
scores_predicted=model1.predict(data)

dict[all_labels[2]] = scores_predicted

data,score=evaluate("final_evaluation_data.csv", all_labels[3])
model1=pickle.load(open("/content/drive/MyDrive/Weights_final/Narrative/bert-base-multilingual-cased_8_2e-05_MSE_3", 'rb'))
scores_predicted=model1.predict(data)

dict[all_labels[3]] = scores_predicted

data,score=evaluate("final_evaluation_data.csv", all_labels[5])
model1=pickle.load(open("/content/drive/MyDrive/Weights_final/Style/bert-base-multilingual-cased_8_2e-05_MSE_2", 'rb'))
scores_predicted=model1.predict(data)

dict[all_labels[5]] = scores_predicted

data,score=evaluate("final_evaluation_data.csv", all_labels[6])
model1=pickle.load(open("/content/drive/MyDrive/Weights_final/Tone/bert-base-multilingual-cased_8_2e-05_MSE_1", 'rb'))
scores_predicted=model1.predict(data)

dict[all_labels[6]] = scores_predicted


In [26]:
data,score=evaluate("final_evaluation_data.csv", all_labels[4])
dict[all_labels[4]] = score

In [6]:
import pandas as pd                                                             #Saving the predicted_scores for all 6 models
df = pd.DataFrame(dict)
df.to_csv('temp.csv')

In [9]:
import numpy as np                                                              #Preparing features for evaluation

df = pd.read_csv("temp.csv")
eval = np.array(df['GEO']).reshape(-1,1)
eval = np.append(eval,np.array(df['ENT']).reshape(-1,1),axis = 1)
eval = np.append(eval,np.array(df['TIME']).reshape(-1,1),axis = 1)
eval = np.append(eval,np.array(df['NAR']).reshape(-1,1),axis = 1)
eval = np.append(eval,np.array(df['STYLE']).reshape(-1,1),axis = 1)
eval = np.append(eval,np.array(df['TONE']).reshape(-1,1),axis = 1)
label = np.array(df['Overall']).reshape(-1,1)
label = label.reshape(-1,)

In [8]:
import torch as T
class Net(T.nn.Module):                                                         #mlp definition
  def __init__(self):
    super(Net, self).__init__()
    self.hid1 = T.nn.Linear(6, 10)  # 8-(10-10)-1
    self.hid2 = T.nn.Linear(10, 10)
    self.oupt = T.nn.Linear(10, 1)

    T.nn.init.xavier_uniform_(self.hid1.weight)
    T.nn.init.zeros_(self.hid1.bias)
    T.nn.init.xavier_uniform_(self.hid2.weight)
    T.nn.init.zeros_(self.hid2.bias)
    T.nn.init.xavier_uniform_(self.oupt.weight)
    T.nn.init.zeros_(self.oupt.bias)

  def forward(self, x):
    z = T.relu(self.hid1(x))
    z = T.relu(self.hid2(z))
    z = self.oupt(z)  # no activation
    return z

In [12]:
import pickle
model = pickle.load(open("NN_model",'rb'))
pred = model(T.tensor(eval.astype(np.float32)))
loss_f = T.nn.MSELoss()
loss = 0
for i in range(len(pred)):
    loss = loss + loss_f(pred[i],T.tensor(label[i])).item()

print(loss/len(pred))
data,score=evaluate("final_evaluation_data.csv", all_labels[4])
model1=pickle.load(open("mbert_model.sav", 'rb'))                #specify model name
pred1=model1.predict(data)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0.1305687193451751


In [12]:
print(len(label))

1680


In [13]:
pred=pred.reshape(-1)
pred1=pred1.reshape(-1)

x1=[]
for i in range(len(pred)):
  x1.append((pred[i]+pred1[i])/2)                                               #Averaging results from current model and mbert output
x=torch.Tensor([x1,df["Overall"]])
coeff=torch.corrcoef(x)
print(coeff)

tensor([[1.0000, 0.7020],
        [0.7020, 1.0000]])
